# Capstone Project 1
## Data Wrangling Steps
The Seeds data set can be found on the UCI ML Repository at https://archive.ics.uci.edu/ml/datasets/seeds. It is simple to download and has a description of the parameters. Upon inspection it is clear that compared to most data sets in use today, this set is very small. There are 210 data points and 8 variables. Seven of these variables are numeric and essentially continuous; they are geometric properties of the seeds taken from precise measurement. The final variable is categorical, and gives the species of the seed with three distinct values: 1="Kama", 2="Rosa", 3="Canadian". The size of the dataset might be one of the challenges to be dealt with. A number of ways to address this have been considered, including cross-validation, bootstrapping, and creating synthetic data.

There are no missing values. There is, however, a mismatch of data in correct columns. After inspecting the original text file, the mismatched columns were easy to see. Some of the entries had, in random places, been shifted over to the right a column or two, leaving blank spaces where the information should be. This misalignment was present in about 5% of the row entries, and and causes an error when attempting to load the file directly into a pandas dataframe. Skipping these rows, the file can be read into a dataframe with 11 missing entries.

An alert giving the rows seen allows us to open the file and write in an initial row that that is beyond the greatest column
of misalignment. Doing this allows all of the data to be read into the file without skipping any rows. The consequence is that there are now extra columns that are full of NaN values. The actual corrupted data can be identified, stored for cleaning,
and removed from the rest of the properly aligned dataframe. 

To align the corrupted data, it is simple to read the correct values into a list in the correct order, leaving out the NaN
entries causing the misalignment. A function herein does this and returns a new dataframe with the data properly aligned and indexed. This is then appended to the uncorrupted dataframe, and sorted back in to the proper order. The result is a tidy dataframe with no missing values and everything properly aligned. The data is ready for analysis in a dataframe called seeds.




In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
#Inspect the raw data:
with open('seeds_raw.txt','r') as file:
    for i in range(10):
        print(file.readline())

15.26	14.84	0.871	5.763	3.312	2.221	5.22	1

14.88	14.57	0.8811	5.554	3.333	1.018	4.956	1

14.29	14.09	0.905	5.291	3.337	2.699	4.825	1

13.84	13.94	0.8955	5.324	3.379	2.259	4.805	1

16.14	14.99	0.9034	5.658	3.562	1.355	5.175	1

14.38	14.21	0.8951	5.386	3.312	2.462	4.956	1

14.69	14.49	0.8799	5.563	3.259	3.586	5.219	1

14.11	14.1	0.8911	5.42	3.302	2.7		5		1

16.63	15.46	0.8747	6.053	3.465	2.04	5.877	1

16.44	15.25	0.888	5.884	3.505	1.969	5.533	1



The text file is tab delimited with some columns having misplaced entries. When attempting to load the file directly into a pandas dataframe, a parsing error occurs. This is easily remdied by adding error_bad_lines=False, causing the corrupted lines to be skipped. A message identifying the skipped rows is also returned:

In [3]:
seeds_df = pd.read_csv('seeds_raw.txt',sep='\t',header=None,error_bad_lines=False)

b'Skipping line 8: expected 8 fields, saw 10\nSkipping line 36: expected 8 fields, saw 10\nSkipping line 61: expected 8 fields, saw 9\nSkipping line 69: expected 8 fields, saw 9\nSkipping line 107: expected 8 fields, saw 9\nSkipping line 136: expected 8 fields, saw 9\nSkipping line 170: expected 8 fields, saw 9\nSkipping line 171: expected 8 fields, saw 9\nSkipping line 173: expected 8 fields, saw 9\nSkipping line 202: expected 8 fields, saw 9\nSkipping line 204: expected 8 fields, saw 9\n'


In [4]:
#A message is returned giving the index position of the displaced rows along with the incorrect number of columns.
#Inspecting this we see that the maximum number of fields seen is ten. 
print('Skipping line 8: expected 8 fields, saw 10\nSkipping line 36: expected 8 fields, saw 10\nSkipping line 61: expected 8 fields, saw 9\nSkipping line 69: expected 8 fields, saw 9\nSkipping line 107: expected 8 fields, saw 9\nSkipping line 136: expected 8 fields, saw 9\nSkipping line 170: expected 8 fields, saw 9\nSkipping line 171: expected 8 fields, saw 9\nSkipping line 173: expected 8 fields, saw 9\nSkipping line 202: expected 8 fields, saw 9\nSkipping line 204: expected 8 fields, saw 9\n')

Skipping line 8: expected 8 fields, saw 10
Skipping line 36: expected 8 fields, saw 10
Skipping line 61: expected 8 fields, saw 9
Skipping line 69: expected 8 fields, saw 9
Skipping line 107: expected 8 fields, saw 9
Skipping line 136: expected 8 fields, saw 9
Skipping line 170: expected 8 fields, saw 9
Skipping line 171: expected 8 fields, saw 9
Skipping line 173: expected 8 fields, saw 9
Skipping line 202: expected 8 fields, saw 9
Skipping line 204: expected 8 fields, saw 9



In [5]:
#However, we are now missing some data. Inspecting the data frame's info shows us we are missing 11 entries. With such a
#small initial data set, this amounts to a significant loss:
print(seeds_df.info(),'\nLoss of',round(11/210 * 100,2),'%')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       199 non-null    float64
 1   1       199 non-null    float64
 2   2       199 non-null    float64
 3   3       199 non-null    float64
 4   4       199 non-null    float64
 5   5       199 non-null    float64
 6   6       199 non-null    float64
 7   7       199 non-null    int64  
dtypes: float64(7), int64(1)
memory usage: 12.6 KB
None 
Loss of 5.24 %


In [6]:
#A quick fix is to add a first row to the dataset that contains the required number of columns or more.
#To do this, we must first read in the data, write the new line, and then write the rest of the data.
#There is no way to directly prepend data to a flat file. 
with open('seeds_raw.txt','r') as original:
    data = original.read()
with open('seeds_raw.txt','w') as mod:
    mod.write("1\t"*10+'\n' + data)

In [7]:
seeds_to_clean = pd.read_csv('seeds_raw.txt',sep='\t',header=None)
seeds_to_clean.head(10)

,0,1,2,3,4,5,6,7,8,9,10
0,1.00,1.00,1.0000,1.000,1.000,1.000,1.000,1.0,1.0,1.0,NaN
1,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1.0,NaN,NaN,NaN
2,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1.0,NaN,NaN,NaN
3,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1.0,NaN,NaN,NaN
4,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1.0,NaN,NaN,NaN
5,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1.0,NaN,NaN,NaN
6,14.38,14.21,0.8951,5.386,3.312,2.462,4.956,1.0,NaN,NaN,NaN
7,14.69,14.49,0.8799,5.563,3.259,3.586,5.219,1.0,NaN,NaN,NaN
8,14.11,14.10,0.8911,5.420,3.302,2.700,NaN,5.0,NaN,1.0,NaN
9,16.63,15.46,0.8747,6.053,3.465,2.040,5.877,1.0,NaN,NaN,NaN


Now all of the data has been read into a dataframe. Next we need only to identify the columns that are misaligned. We can drop the first row, then look at the dataframe's info.

In [8]:
seeds_to_clean.drop(0,axis=0,inplace=True)
seeds_to_clean.reset_index(drop=True,inplace=True)
print(seeds_to_clean.head())
seeds_to_clean.info()

      0      1       2      3      4      5      6    7   8   9   10
0  15.26  14.84  0.8710  5.763  3.312  2.221  5.220  1.0 NaN NaN NaN
1  14.88  14.57  0.8811  5.554  3.333  1.018  4.956  1.0 NaN NaN NaN
2  14.29  14.09  0.9050  5.291  3.337  2.699  4.825  1.0 NaN NaN NaN
3  13.84  13.94  0.8955  5.324  3.379  2.259  4.805  1.0 NaN NaN NaN
4  16.14  14.99  0.9034  5.658  3.562  1.355  5.175  1.0 NaN NaN NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       210 non-null    float64
 1   1       210 non-null    float64
 2   2       207 non-null    float64
 3   3       210 non-null    float64
 4   4       209 non-null    float64
 5   5       210 non-null    float64
 6   6       206 non-null    float64
 7   7       206 non-null    float64
 8   8       10 non-null     float64
 9   9       2 non-null      float64
 10  10      0 non-null      float64


The info above now tells us which columns have misalignments, and how many. Column 10 has only NaN entries, so it can be dropped.

In [9]:
seeds_to_clean.drop(10,axis=1,inplace=True)
seeds_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       210 non-null    float64
 1   1       210 non-null    float64
 2   2       207 non-null    float64
 3   3       210 non-null    float64
 4   4       209 non-null    float64
 5   5       210 non-null    float64
 6   6       206 non-null    float64
 7   7       206 non-null    float64
 8   8       10 non-null     float64
 9   9       2 non-null      float64
dtypes: float64(10)
memory usage: 16.5 KB


Now we can look at the columns individually with the info as a guide.

In [10]:
#Here we create a dataframe to hold the corrupted data, then cycle through the full raw data looking for NaN values.
#When found, we add the rows to which these values exist for later correction. We can simultaneously delete these
#vagrant rows from the rest of the raw data:
corrupt_data = pd.DataFrame(columns=seeds_to_clean.columns)
for column in seeds_to_clean.columns:
    if column > 7:
        break
    for k,v in enumerate(seeds_to_clean[column].isna()):
        if v==True:
            if k not in corrupt_data.index:
                temp = pd.Series(seeds_to_clean.iloc[k,:])
                corrupt_data = corrupt_data.append(temp)
            
corrupt_data.sort_index(inplace=True)
corrupt_data

,0,1,2,3,4,5,6,7,8,9
7,14.11,14.10,0.8911,5.4200,3.302,2.700,NaN,5.000,NaN,1.0
35,16.12,15.00,NaN,0.9000,NaN,5.709,3.485,2.270,5.443,1.0
60,11.42,12.86,0.8683,5.0080,2.850,2.700,NaN,4.607,1.000,NaN
68,14.37,14.39,0.8726,5.5690,3.153,1.464,5.300,NaN,1.000,NaN
106,18.85,16.17,0.9056,6.1520,3.806,2.843,6.200,NaN,2.000,NaN
135,15.38,14.66,0.8990,5.4770,3.465,3.600,NaN,5.439,2.000,NaN
169,11.24,13.00,NaN,0.8359,5.090,2.715,3.521,5.088,3.000,NaN
170,11.02,13.00,NaN,0.8189,5.325,2.701,6.735,5.163,3.000,NaN
172,11.27,12.97,0.8419,5.0880,2.763,4.309,5.000,NaN,3.000,NaN
201,12.67,13.32,0.8977,4.9840,3.135,2.300,NaN,4.745,3.000,NaN


In [11]:
#Here we use the index of corrupted data to remove those rows from the main dataset:
seeds_to_clean.drop(corrupt_data.index,axis=0,inplace=True)
seeds_to_clean.drop([8,9],axis=1,inplace=True)
seeds_to_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 209
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       199 non-null    float64
 1   1       199 non-null    float64
 2   2       199 non-null    float64
 3   3       199 non-null    float64
 4   4       199 non-null    float64
 5   5       199 non-null    float64
 6   6       199 non-null    float64
 7   7       199 non-null    float64
dtypes: float64(8)
memory usage: 14.0 KB


Now we have a dataframe containing the corrupted data, and have removed this data from the raw dataframe. The unnecessarry columns have aslo been removed, leaving a tidy dataframe with no missing values. Next we must deal witht he corrupted data, aligning the wayward entries. Then we will add this data back to the original dataframe, giving us a complete set of data in a tidy dataframe that is ready for analysis.

In [12]:
#Here we create a reusable function to clean other misaligned datasets. 
def aligner(df):
    """Takes a dataframe with misaligned values having been misplaced by NaN.
    The function iterates through the values and creates a new dataframe with the
    properly aligned values in their correct columns."""
    lst= []
    for i,j in corrupt_data.iterrows():
        tmp=[k for k in j if np.isnan(k)==False]
        lst.append(tmp)
    aligned=pd.DataFrame(lst,index=df.index)
    return(aligned)

In [13]:
#Using the aligner function on the corrupt data returns a properly indexed 
#dataframe with all values in their proper columns.
aligned = aligner(corrupt_data)
aligned

,0,1,2,3,4,5,6,7
7,14.11,14.10,0.8911,5.420,3.302,2.700,5.000,1.0
35,16.12,15.00,0.9000,5.709,3.485,2.270,5.443,1.0
60,11.42,12.86,0.8683,5.008,2.850,2.700,4.607,1.0
68,14.37,14.39,0.8726,5.569,3.153,1.464,5.300,1.0
106,18.85,16.17,0.9056,6.152,3.806,2.843,6.200,2.0
135,15.38,14.66,0.8990,5.477,3.465,3.600,5.439,2.0
169,11.24,13.00,0.8359,5.090,2.715,3.521,5.088,3.0
170,11.02,13.00,0.8189,5.325,2.701,6.735,5.163,3.0
172,11.27,12.97,0.8419,5.088,2.763,4.309,5.000,3.0
201,12.67,13.32,0.8977,4.984,3.135,2.300,4.745,3.0


In [16]:
#Finally, we can append the aligned data to the rest of the good data,
#creating a master dataframe called seeds, that is ready for analysis.
columns = ['area','perimeter','compactness','length','width','asymmetry_coefficient','groove_length','class']
seeds = pd.concat([seeds_to_clean,aligned])
seeds.sort_index(axis=0,inplace=True)
seeds.columns=columns
seeds

,area,perimeter,compactness,length,width,asymmetry_coefficient,groove_length,class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1.0
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1.0
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1.0
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1.0
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1.0
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3.0
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3.0
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3.0
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3.0


In [17]:
seeds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 0 to 209
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   area                   210 non-null    float64
 1   perimeter              210 non-null    float64
 2   compactness            210 non-null    float64
 3   length                 210 non-null    float64
 4   width                  210 non-null    float64
 5   asymmetry_coefficient  210 non-null    float64
 6   groove_length          210 non-null    float64
 7   class                  210 non-null    float64
dtypes: float64(8)
memory usage: 14.8 KB
